# Binary Search

### Goal
Find an element in a list

- list must be **sorted**
- list may be empty

### Template

In [8]:
def binary_search(lst, n):
    """
    Given a sorted list and an integer, find out if the integer is in the list
    
    @param
    lst: List[int]
    n: int
    
    @return bool
    """

### Examples

In [24]:
assert binary_search([1, 2, 5], 5) == True

# list with an odd number of elements
assert binary_search([1, 2, 3, 5, 6], 5) == True

# list with an even number of elements
assert binary_search([1, 5, 7, 8, 9, 10], 5) == True

### Edge Cases

In [23]:
# empty list
assert binary_search([], 5) == False

# list with one element
assert binary_search([1], 5) == False
assert binary_search([5], 5) == True

# list with two elements
assert binary_search([1, 5], 1) == True
assert binary_search([1, 5], 5) == True
assert binary_search([1, 5], 3) == False

### Idea 1

Find the middle index m of the list L and compare L[m] with n

- If L[m] == n, return true
- If L[m] > n, n must be in the first half of the list because the list is sorted. Search in the first half (recursively).
- If L[m] < n, n must be in the second half of the list. Search in the second half (recursively).

How to find m?
- m = len(L) // 2 (Remember // means integer division in Python)
- Examples:
    - len(L) = 1, m = 0
    - len(L) = 2, m = 1
    - len(L) = 0, m = 0. L[m] will raise an error, and so this edge case need be handled.

How to get the first/second half of the list?
- First half = L[:m]. Remember that this is right exclusive in Python. i.e. [L[0], L[1], ..., L[m-1]]
- Second half = L[m+1:]

### Precondition
- List must be sorted
- List may be empty

### Invariant
- We rule out a half of the list for each iteration.

### Postcondition
- If the final list has two elements, m = 1. No matter we rule out the first or second half, L[:1] has one element, and L[2:] has no element. We will fall into the case when L has zero or one element.
- If the final list has one element, m = 0. We will check for L[0] == n. L[:0] and L[1:] both have zero element and so we will fall into the case when L has zero element.
- If the final list has zero element, we handle this as an edge case specially.

Therefore, it is the base case for our recursion when the list is empty.

### Termination
Based on our invariant and postcondition analysis, after each iteration the list becomes shorter. Thus, our algorithm will terminate eventually.

In [13]:
def binary_search(lst, n):   
    # check for empty
    if not lst:
        return False
    
    m = len(lst) // 2
    
    if lst[m] == n:
        return True
    elif lst[m] > n:
        return binary_search(lst[:m], n)
    elif lst[m] < n:
        return binary_search(lst[m+1:], n)

### Optimization 1
Recursion consumes memory stack. Can we avoid using recursion?

We can use a while loop as below.

In [21]:
def binary_search(lst, n):
    # @precondition: lst may be empty or not
    
    while lst:
        # @invariant: lst must not be empty in the while loop
        m = len(lst) // 2

        if lst[m] == n:
            return True
        elif lst[m] > n:
            lst = lst[:m]
        elif lst[m] < n:
            lst = lst[m+1:]
    
    # @postcondition: lst is empty
    return False

### Optimization 2

The slicing operation of list in Python always creates a new list, which consumes extra memory. Can we avoid slicing?

We can try to remember the start and end indices of the sublist as below.

In [2]:
def binary_search(lst, n):
    start = 0
    end = len(lst) # end exclusive
    
    while start < end:
        m = (start + end) // 2

        if lst[m] == n:
            return True
        elif lst[m] > n:
            # move the end index to m.
            end = m
        elif lst[m] < n:
            # move the start index to m + 1.
            start = m + 1
    
    # @postcondition: lst is empty
    return False

# Binary Search for the First Occurance

### Goal
Find the first occurance of an element in a sorted list.

- There may be multiple same elements.
- Return the index of the first occurance.
- Return -1 if not found.

### Examples

In [36]:
assert binary_search([1,2,2,3,3,3,5], 3) == 3
assert binary_search([1,2,2,3,3,3,5], 2) == 1
assert binary_search([1,2,2,3,3,3,5], 6) == -1

### Edge Cases

In [37]:
assert binary_search([], 2) == -1
assert binary_search([2], 2) == 0
assert binary_search([2,2,2,2,2,2,2,2,2], 2) == 0

### Idea 2
Same as before, but with some minor changes:

Find the middle index m of the list L.
- If L[m] == n, we also check for the previous number L[m-1]. If L[m-1] != n, return m. Otherwise, the target must be in the first half of the list and so continue search in the first half.
    - Notice that we have a special case: m = 0, that we must handle. Because L[m] is already the first element of the list, return m.
- If L[m] > n, search in the first half of the list.
- If L[m] < n, search in the second half of the list.

In [10]:
def binary_search(lst, n):
    start = 0
    end = len(lst)
    
    while start < end:
        m = (start + end) // 2
        
        if lst[m] == n:
            # Notice: we must handle m = 0
            if m == 0 or lst[m-1] != n:
                return m
            else:
                # @invariant: lst[m-1] must be == n
                # search the first half
                end = m
        elif lst[m] > n:
            end = m
        else:
            start = m + 1

    return -1

### Idea 3

Find the middle index m of the list L.
- If L[m] == n, we search **backwards** back from L[m-1], L[m-2], ... until we find the first occurance)
- If L[m] > n, search in the first half of the list.
- If L[m] < n, search in the second half of the list.

How do we search backwards?

- A naive approach will be linear search (search backwards one by one). However, the worst case will have O(N) time complexity.
- An optimal approach is "skip search". The intuition behind this approach is to "jump" when go backwards by doubling the skip distance, so that we can reach the target more quickily.

In [38]:
def binary_search(lst, n):
    start = 0
    end = len(lst)
    
    while start < end:
        m = (start + end) // 2
        
        if lst[m] == n:
            return skip_search(lst, n, m)
        elif lst[m] > n:
            end = m
        else:
            start = m + 1

    return -1

### recursive solution

In [39]:
def skip_search(lst, n, m):
    # @precondition: lst[m] == n and m >= 0
    
    # @edge case
    if m == 0:
        return m
    
    # @base case: d = 1
    if lst[m - 1] != n:
        return m
    
    d = 2
    while lst[m - d] == n and m - d >= 0:
        d = d * 2
    
    # @postcondition: lst[m - d/2] == n and m - d / 2 >= 0
    
    return skip_search(lst, n, m - d / 2)

### iterative solution

In [40]:
def skip_search(lst, n, m):
    # @precondition: lst[m] == n and m >= 0
    
    while m != 0:
        # @invariant: lst[m] == n and m > 0 (m - 1 >= 0)
        if lst[m - 1] != n:
            return m
        
        d = 2
        while lst[m - d] == n and m - d >= 0:
            d = d * 2
        
        # @postcondition: lst[m - d / 2] == n and m - d / 2 >= 0
        m = m - d / 2
    
    # @postcondition: m == 0
    return m

# Binary Search for Both the First and Last Occurance

After slightly changing the code of Idea 2, it's trivial to build a binary search function to find the last ocurrance.
We need to run the binary search twice over the whole array.

With Idea 3, we run binary search once to find the position where L[m] = n, and then call skip search twice to find both the first and last occurance.

In this case, Idea 3 will be slightly better than Idea 2 because 1) we do not to run binary search twice 2) skip search only need search in a smaller space in general.